# Project 11: Index Tracking and Forward Testing (US Stocks)

## Importing the Data

In [ ]:
import pandas as pd

In [ ]:
const = pd.read_csv("const_prices.csv", parse_dates = ["Date"], index_col = "Date")
const

In [ ]:
const.info()

In [ ]:
dji = pd.read_csv("dji.csv", parse_dates = ["Date"], index_col = "Date")
dji

In [ ]:
dji.info()

In [ ]:
dji = dji.Close.to_frame().copy()

In [ ]:
dji

In [ ]:
df = pd.concat([const, dji], axis = 1)
df

In [ ]:
df.rename(columns = {"Close": "DJI"}, inplace = True)

In [ ]:
more_info = pd.read_csv("const.csv")
more_info

## Transforming the Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use("seaborn")

In [ ]:
df.plot(figsize = (20, 12), fontsize = 15, secondary_y = "DJI")
plt.show()

In [ ]:
df.div(df.iloc[0]).mul(100)

In [ ]:
norm = df.div(df.iloc[0]).mul(100)
norm

In [ ]:
norm.plot(figsize = (20, 12), fontsize = 15)
plt.legend(fontsize = 12)
plt.show()

In [ ]:
norm.drop(columns =["AAPL"]).plot(figsize = (20, 12), fontsize = 15)
plt.legend(fontsize = 12)
plt.show()

In [ ]:
ret = df.pct_change()

In [ ]:
ret

In [ ]:
ret.describe()

In [ ]:
def summary_ann(returns):
    summary = returns.agg(["mean", "std"]).T
    summary["Return"] = summary["mean"] * 252
    summary["Risk"] = summary["std"] * np.sqrt(252)
    summary.drop(columns = ["mean", "std"], inplace = True)
    return summary

In [ ]:
summary = summary_ann(ret)
summary

In [ ]:
summary = summary.merge(more_info.loc[:, ["Ticker", "Company"]],
                        how = "left", left_index= True, right_on = "Ticker")

In [ ]:
summary.set_index("Company", inplace = True)

In [ ]:
summary

In [ ]:
summary.rename(index = {np.nan : "Dow Jones"}, inplace = True)

In [ ]:
summary

## Explanatory Data Analysis

In [ ]:
ret.info()

In [ ]:
summary2 = summary.drop(index = "Dow Inc.")
summary2

In [ ]:
summary2.sort_values(by = "Return", ascending = False).plot(kind = "bar", figsize = (20, 12), 
                                                            fontsize = 15)
plt.legend(fontsize = 15)
plt.show()

In [ ]:
summary2.sort_values(by = "Risk", ascending = False).plot(kind = "bar", figsize = (20, 12), 
                                                          fontsize = 15)
plt.legend(fontsize = 15)
plt.show()

In [ ]:
summary2.set_index("Ticker", inplace = True)

In [ ]:
summary2

In [ ]:
summary2.plot(kind = "scatter", x = "Risk", y = "Return", figsize = (15,12), s = 50, fontsize = 15)
for i in summary2.index:
    plt.annotate(i, xy=(summary2.loc[i, "Risk"]+0.002, summary2.loc[i, "Return"]+0.002), size = 15)
plt.xlabel("ann. Risk(std)", fontsize = 15)
plt.ylabel("ann. Return", fontsize = 15)
plt.title("Risk/Return", fontsize = 20)
plt.show()

In [ ]:
corr_matrix = ret.corr()
corr_matrix

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize = (20, 12))
sns.heatmap(corr_matrix, cmap='RdYlGn',annot=True, vmin = 0.2, vmax = 0.8)
plt.show()

## Index Tracking - an Introduction

Plan: Track the DJI with only __10 Stocks__ (Tracking Portfolio).

Criteria to select Tracking Portfolio: Those 10 Stocks with the __lowest individual Tracking Error__ (TE) in the Training/Fitting Period.

- Training/Fitting Period: 2017 & 2018
- (Forward) Testing Period: 2019

Once we have identified the 10 Tracking Stocks we need to find their __optimal weights__ in the Tracking Portfolio that __minimize TE__.

In [ ]:
df.info()

In [ ]:
df2 = df.loc["2017":"2018"].copy()
df2

In [ ]:
norm2 = df2.div(df2.iloc[0]).mul(100)
norm2

In [ ]:
ret2 = ret.loc["2017":"2018"].copy()
ret2

In [ ]:
ret_diff = ret2.sub(ret2.DJI, axis = 0)
ret_diff

In [ ]:
ret_diff.agg(["mean", "std"]).T

## Selecting the Tracking Stocks

In [ ]:
def tracking_ann(returns):
    summary = returns.agg(["mean", "std"]).T
    summary["Premium"] = summary["mean"] * 252
    summary["TE"] = summary["std"] * np.sqrt(252)
    summary.drop(columns = ["mean", "std"], inplace = True)
    return summary

In [ ]:
tracking = tracking_ann(ret_diff)
tracking

In [ ]:
tracking.sort_values(by = "TE")

In [ ]:
noa = 10

In [ ]:
tracking_stocks = tracking.sort_values(by = "TE").index[:noa+1]

In [ ]:
tracking_stocks

In [ ]:
norm_track = norm2.loc[:, tracking_stocks]
norm_track

In [ ]:
norm_track.plot(figsize = (20, 12), fontsize = 15)
plt.legend(fontsize = 12)
plt.show()

In [ ]:
tracking_stocks = tracking_stocks[1:]
tracking_stocks

## Index Tracking - A simple Tracking Portfolio

In [ ]:
tracking_stocks

In [ ]:
#equal weights (starting point of optimization)
eweights = np.full(noa, 1/noa)
eweights

In [ ]:
ret2.loc[:, tracking_stocks].mean(axis = 1)

In [ ]:
ret2.loc[:, tracking_stocks].dot(eweights)

In [ ]:
#calculate portfolio returns (based on weights)
def port_ret(weights):
    return ret2.loc[:, tracking_stocks].dot(weights)

In [ ]:
#define function to be minimized -> minimize tracking error of tracking portfolio
def tracking_error(weights): 
     return port_ret(weights).sub(ret2.DJI).std() * np.sqrt(252)

In [ ]:
tracking_error(eweights)

In [ ]:
ew_port = port_ret(eweights)[1:].add(1).cumprod().mul(100)
ew_port

In [ ]:
norm_track["EW_Port"] = ew_port
norm_track.iloc[0, -1] = 100

In [ ]:
norm_track

In [ ]:
norm_track[["EW_Port", "DJI"]].plot(figsize = (15, 10))
plt.legend(fontsize = 20, loc = 2)
plt.show()

## Index Tracking - The optimal Tracking Portfolio

In [ ]:
import scipy.optimize as sco #import scipy optimize

In [ ]:
#constraint: weights must sum up to 1 -> sum of weights - 1 = 0
cons = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})
cons

In [ ]:
#bounds: all weights shall be between 0 and 1 -> can be changed
bnds =  tuple((0,1) for x in range(noa))
bnds

In [ ]:
#run optimization based on function to be minimized, starting with equal weights and based on respective bounds and constraints
opts = sco.minimize(tracking_error, eweights, method = "SLSQP", bounds = bnds, constraints= cons)

In [ ]:
#output of optimization
opts

In [ ]:
#getting the optimal weights
optimal_weights = opts["x"]
optimal_weights

In [ ]:
pd.Series(index = tracking_stocks, data = optimal_weights)

In [ ]:
opt_port = port_ret(optimal_weights)[1:].add(1).cumprod().mul(100)
opt_port

In [ ]:
norm_track["Opt_Port"] = opt_port
norm_track.iloc[0, -1] = 100

In [ ]:
norm_track

In [ ]:
norm_track[["Opt_Port", "DJI"]].plot(figsize = (15, 10))
plt.legend(fontsize = 20, loc = 2)
plt.show()

In [ ]:
summary_ann(norm_track[["Opt_Port", "EW_Port", "DJI"]].pct_change())

## Forward Testing (Part 1)

In [ ]:
df

In [ ]:
ret

In [ ]:
tracking_stocks

In [ ]:
def tracking_error_gen(data, tracking_stocks, weights, index, start, end):
    return data.loc[start:end, tracking_stocks].dot(weights).sub(ret.loc[start:end, index]).std() * np.sqrt(252)   

In [ ]:
def visual(data, tracking_stocks, weights, index, start, end):
    track = data.loc[start:end, tracking_stocks].dot(weights).add(1).cumprod().mul(100)
    track.name = "Portfolio"
    index = data.loc[start:end, index].add(1).cumprod().mul(100)
    plt.figure(figsize = (12,8))
    track.plot()
    index.plot()
    plt.legend(fontsize = 20, loc = 2)
    plt.show()

In [ ]:
tracking_error_gen(ret, tracking_stocks, optimal_weights, "DJI", "2017-01-01", "2018-12-31")

In [ ]:
forward_te = tracking_error_gen(ret, tracking_stocks, optimal_weights, "DJI", 
                                "2019-01-01", "2019-12-31")
forward_te

In [ ]:
visual(ret, tracking_stocks, optimal_weights, "DJI", "2019-01-01", "2019-12-31")

## Forward Testing (Part 2)

In [ ]:
bucket = df.columns.drop(["DOW", "DJI"])
bucket

In [ ]:
np.random.choice(bucket, size = noa, replace = False)

In [ ]:
random_numbers = np.random.random(noa)
random_weights = random_numbers / random_numbers.sum()
random_weights.sum()

In [ ]:
np.random.seed(123)
random_stocks = np.random.choice(bucket, size = noa, replace = False)
random_numbers = np.random.random(noa)
random_weights = random_numbers / random_numbers.sum()

In [ ]:
print(random_stocks, random_weights)

In [ ]:
tracking_error_gen(ret, random_stocks, random_weights, "DJI", "2019-01-01", "2019-12-31")

In [ ]:
forward_te

In [ ]:
visual(ret, random_stocks, random_weights, "DJI", "2019-01-01", "2019-12-31")

In [ ]:
tes = np.empty(10000)
np.random.seed(123)
for i in range(10000):
    random_numbers = np.random.random(noa)
    random_weights = random_numbers / random_numbers.sum()
    random_stocks = np.random.choice(bucket, size = noa, replace = False)
       
    tes[i] = tracking_error_gen(ret, random_stocks, random_weights, "DJI", "2019-01-01", "2019-12-31")

In [ ]:
(tes < forward_te).mean()

In [ ]:
plt.figure(figsize = (12, 8))
plt.hist(tes, bins = 100)
plt.show()